In [81]:

# import os
# import asyncio
# import pickle
# import pandas as pd
# import streamlit as st
# from dotenv import load_dotenv

# # LangChain integrations
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
# from langchain.agents import initialize_agent, Tool

# # --------------------------
# # Setup
# # --------------------------
# load_dotenv()
# st.title("📊 Marketing Assistant with Churn Prediction")

# google_api_key = os.getenv("GOOGLE_API_KEY")
# chat_model_name = "models/gemini-2.0-flash-lite-preview"

# # Ensure event loop exists
# try:
#     asyncio.get_running_loop()
# except RuntimeError:
#     asyncio.set_event_loop(asyncio.new_event_loop())

# # --------------------------
# # Load your churn model
# # --------------------------
# with open(r"D:\ITI\1_etisalt\Etisalat\churn\models\churn.pkl", "rb") as f:
#     churn_model = pickle.load(f)

# # Define churn prediction tool
# def predict_churn(data: dict) -> str:
#     """
#     Takes a dict of customer features and returns churn prediction.
#     Expected fields:
#     CustomerId, Gender, Senior_Citizen, Is_Married, Dependents, tenure,
#     Phone_Service, Dual, Internet_Service, Online_Security, Online_Backup,
#     Device_Protection, Tech_Support, Streaming_TV, Streaming_Movies,
#     Contract, Paperless_Billing, Payment_Method, Monthly_Charges, Total_Charges
#     """
#     try:
#         df = pd.DataFrame([data])
#         pred = churn_model.predict(df)[0]
#         prob = None
#         if hasattr(churn_model, "predict_proba"):
#             prob = churn_model.predict_proba(df)[0].max()

#         if prob:
#             return f"Prediction: {pred} (confidence: {prob:.2f})"
#         return f"Prediction: {pred}"
#     except Exception as e:
#         return f"❌ Error making prediction: {str(e)}"

# # Wrap model as tool
# tools = [
#     Tool(
#         name="Churn Predictor",
#         func=predict_churn,
#         description=(
#             "Use this tool to predict if a customer will churn. "
#             "Provide customer details as a JSON dict with keys such as "
#             "Gender, Senior_Citizen, Is_Married, Dependents, tenure, "
#             "Phone_Service, Internet_Service, Contract, Monthly_Charges, "
#             "Total_Charges, etc."
#         )
#     )
# ]

# # --------------------------
# # Session state for messages
# # --------------------------
# if "messages" not in st.session_state:
#     st.session_state.messages = []
#     st.session_state.messages.append(SystemMessage(
#         "You are a marketing assistant. "
#         "If the user asks about churn prediction, use the Churn Predictor tool. "
#         "Otherwise, answer normally."
#     ))

# # --------------------------
# # Display previous messages
# # --------------------------
# for message in st.session_state.messages:
#     if isinstance(message, HumanMessage):
#         with st.chat_message("user"):
#             st.markdown(message.content)
#     elif isinstance(message, AIMessage):
#         with st.chat_message("assistant"):
#             st.markdown(message.content)

# # --------------------------
# # Chat input
# # --------------------------
# prompt = st.chat_input("Ask about churn or marketing insights...")

# if prompt:
#     # Add user input
#     with st.chat_message("user"):
#         st.markdown(prompt)
#     st.session_state.messages.append(HumanMessage(prompt))

#     # Create LLM + Agent
#     model_llm = ChatGoogleGenerativeAI(
#         model=chat_model_name,
#         temperature=0,
#         google_api_key=google_api_key
#     )

#     agent = initialize_agent(
#         tools=tools,
#         llm=model_llm,
#         agent="zero-shot-react-description",
#         verbose=True
#     )

#     # Run agent
#     result = agent.run(prompt)

#     # Display response
#     with st.chat_message("assistant"):
#         st.markdown(result)
#     st.session_state.messages.append(AIMessage(result))


In [82]:
import os
import asyncio
import pickle
import pandas as pd
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.agents import initialize_agent, Tool

In [83]:

load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
chat_model_name = "models/gemini-2.0-flash-lite-preview"


In [84]:
# Ensure event loop exists
try:
    asyncio.get_running_loop()
except RuntimeError:
    asyncio.set_event_loop(asyncio.new_event_loop())


with open(r"D:\ITI\1_etisalt\Etisalat\churn\models\churn.pkl", "rb") as f:
    churn_model = pickle.load(f)

In [85]:
def predict_churn(data: dict) -> str:
    """
    Takes a dict of customer features and returns churn prediction.
    """
    try:
        one_raw = pd.DataFrame([data])
        pred = churn_model.predict(one_raw)[0]
        prob = None
        if hasattr(churn_model, "predict_proba"):
            prob = churn_model.predict_proba(one_raw)[0].max()

        if prob:
            return f"Prediction: {pred} (confidence: {prob:.2f})"
        return f"Prediction: {pred}"
    except Exception as e:
        return f"❌ Error making prediction: {str(e)}"


In [86]:
global_df = r'D:\ITI\1_etisalt\Etisalat\churn\data\raw\WA_Fn-UseC_-Telco-Customer-Churn.csv'


def summary_stats(df) -> str:
    try:
        # If it's a CSV path or JSON string
        if isinstance(df, str):
            if df.endswith(".csv"):
                df = pd.read_csv(df)
            else:
                # Try parsing JSON string
                import json
                df = pd.DataFrame(json.loads(df))
        
        # If it's already a DataFrame
        if isinstance(df, pd.DataFrame):
            return df.describe(include="all").to_string()
        else:
            return "❌ Input must be a DataFrame, CSV path, or JSON string."
    except Exception as e:
        return f"❌ Error in summary_stats: {str(e)}"


In [87]:
stats_tool = Tool(
    name="Summary Stats",
    func=lambda _: summary_stats(global_df),
    description="Returns descriptive statistics for numerical and categorical features."
)

churn_predic_tool = Tool(
        name="Churn Predictor",
        func=predict_churn,
        description=(
            "Use this tool to predict if a customer will churn. "
            "Provide customer details as a JSON dict with keys such as "
            "Gender, Senior_Citizen, Is_Married, Dependents, tenure, "
            "Phone_Service, Internet_Service, Contract, Monthly_Charges, "
            "Total_Charges, etc."
        )
    )


In [88]:
tools = [
    churn_predic_tool,
    stats_tool
]

In [89]:
model_llm = ChatGoogleGenerativeAI(
    model=chat_model_name,
    temperature=0,
    google_api_key=google_api_key
)

agent = initialize_agent(
    tools=tools,
    llm=model_llm,
    agent="zero-shot-react-description",
    verbose=True,
    handle_parsing_errors=True

)


In [96]:
query = """Predict churn for the following customer: {
  "Senior_Citizen": 0,
  "Is_Married": "Yes",
  "Dependents": "No",
  "tenure": 5,
  "Internet_Service": "DSL",
  "Online_Security": "No",
  "Online_Backup": "Yes",
  "Device_Protection": "No",
  "Tech_Support": "No",
  "Streaming_TV": "Yes",
  "Streaming_Movies": "No",
  "Contract": "Month-to-month",
  "Paperless_Billing": "Yes",
  "Payment_Method": "Electronic check",
  "Monthly_Charges": 75.0,
  "Total_Charges": 350.0
}"""
result = agent.run(query)
print("Assistant:", result)




> Entering new AgentExecutor chain...
I need to predict churn for the given customer data. I should use the Churn Predictor tool.
Action: Churn Predictor
Action Input: {"Senior_Citizen": 0, "Is_Married": "Yes", "Dependents": "No", "tenure": 5, "Internet_Service": "DSL", "Online_Security": "No", "Online_Backup": "Yes", "Device_Protection": "No", "Tech_Support": "No", "Streaming_TV": "Yes", "Streaming_Movies": "No", "Contract": "Month-to-month", "Paperless_Billing": "Yes", "Payment_Method": "Electronic check", "Monthly_Charges": 75.0, "Total_Charges": 350.0}
Observation: ❌ Error making prediction: 'NoneType' object is not iterable
Thought:I apologize, but I am unable to fulfill this request. The Churn Predictor tool returned an error. I am unable to proceed without a successful prediction.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I apologize, but I am unable to fulfill this request. The Churn Predictor tool returned an error. I am unable to proceed without

In [91]:
data = {
    "CustomerId": "3668-QPYBK",
    "Gender": "Male",
    "Senior_Citizen": 0,
    "Is_Married": "No",
    "Dependents": "No",
    "tenure": 2,
    "Phone_Service": "Yes",
    "Dual": "No",
    "Internet_Service": "DSL",
    "Online_Security": "Yes",
    "Online_Backup": "Yes",
    "Device_Protection": "No",
    "Tech_Support": "No",
    "Streaming_TV": "No",
    "Streaming_Movies": "No",
    "Contract": "Month-to-month",
    "Paperless_Billing": "Yes",
    "Payment_Method": "Mailed check",
    "Monthly_Charges": 53.85,
    "Total_Charges": 108.15
}


In [92]:

import pandas as pd

one_raw = pd.DataFrame([data])
pred = churn_model.predict(one_raw)[0]

prob = churn_model.predict_proba(one_raw)[0].max()

print(f"Prediction: {pred} (confidence: {prob:.2f})")

Prediction: 0 (confidence: 0.79)


In [93]:
print(churn_model.named_steps["preprocessor"].feature_names_in_)


['Senior_Citizen ' 'Is_Married' 'Dependents' 'tenure' 'Internet_Service'
 'Online_Security' 'Online_Backup' 'Device_Protection' 'Tech_Support'
 'Streaming_TV' 'Streaming_Movies' 'Contract' 'Paperless_Billing'
 'Payment_Method' 'Monthly_Charges' 'Total_Charges']


In [94]:
query = "provide data summury"

result = agent.run(query)
print("Assistant:", result)




> Entering new AgentExecutor chain...
I need to get summary statistics for the customer data. I will use the Summary Stats tool.
Action: Summary Stats
Action Input: _
Observation:         customerID gender  Senior_Citizen  Is_Married Dependents       tenure Phone_Service  Dual Internet_Service Online_Security Online_Backup Device_Protection Tech_Support Streaming_TV Streaming_Movies        Contract Paperless_Billing    Payment_Method  Monthly_Charges Total_Charges Churn
count         7043   7043      7043.000000       7043       7043  7043.000000          7043  7043             7043            7043          7043              7043         7043         7043             7043            7043              7043              7043      7043.000000          7043  7043
unique        7043      2              NaN          2          2          NaN             2     3                3               3             3                 3            3            3                3               3       

In [95]:


print(summary_stats(global_df))

        customerID gender  Senior_Citizen  Is_Married Dependents       tenure Phone_Service  Dual Internet_Service Online_Security Online_Backup Device_Protection Tech_Support Streaming_TV Streaming_Movies        Contract Paperless_Billing    Payment_Method  Monthly_Charges Total_Charges Churn
count         7043   7043      7043.000000       7043       7043  7043.000000          7043  7043             7043            7043          7043              7043         7043         7043             7043            7043              7043              7043      7043.000000          7043  7043
unique        7043      2              NaN          2          2          NaN             2     3                3               3             3                 3            3            3                3               3                 2                 4              NaN          6531     2
top     3186-AJIEK   Male              NaN         No         No          NaN           Yes    No      Fiber optic 